In [7]:
import pandas as pd
import numpy as np
import utils
from TradingAgent import PairsTradingAgent

In [8]:
raw_data = pd.read_csv("../data/raw_data.csv", index_col=0)
raw_data.index = pd.to_datetime(raw_data.index)
raw_data.head()

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,40.689999,40.910000,26.337500,57.610001,42.930000,22.950001,101.830002,91.970001,54.439999,35.840000,...,124.980133,29.83,68.769997,35.700001,77.459999,36.080002,51.912292,98.844658,66.489998,47.270000
2016-01-05,40.549999,40.520000,25.677500,57.369999,42.919998,23.033333,102.360001,92.339996,54.040001,36.240002,...,125.839996,29.90,70.070000,36.060001,78.120003,36.070000,51.782890,100.902916,64.820000,48.009998
2016-01-06,40.730000,41.230000,25.174999,57.380001,42.560001,23.070000,102.160004,91.019997,51.740002,35.360001,...,125.839996,29.24,66.440002,36.439999,77.470001,35.619999,51.416248,101.339806,62.230000,48.020000
2016-01-07,39.000000,40.450001,24.112499,57.209999,41.540001,23.046667,99.160004,89.110001,50.419998,34.520000,...,114.949997,28.16,60.189999,36.580002,76.230003,34.700001,49.662113,99.009712,59.410000,46.560001
2016-01-08,38.590000,40.369999,24.240000,55.650002,40.669998,22.806667,98.199997,87.849998,49.980000,34.389999,...,116.620003,27.90,57.740002,36.180000,74.690002,34.369999,48.986340,98.592232,59.250000,45.880001


In [9]:
trading_pairs = pd.read_csv("../trading_pairs.csv", index_col=0)
sorted_trading_pairs = trading_pairs.sort_values("best_return", ascending=False)
sorted_trading_pairs.head()

,leg1,leg2,coint_t,coint_pvalue,hedge_ratio,adf_pvalue,hurst_exponent,half_life,zero_cross,best_threshold,best_return
901,DUK,ENPH,-3.922743,0.009225,7.188884,0.006725,0.409722,52.154031,111,0.95,112.380872
742,CZR,ETSY,-4.411852,0.001703,2.288153,0.006295,0.440650,57.508409,60,1.60,94.771411
159,ALGN,CZR,-5.141951,0.000089,0.147969,0.000114,0.373032,34.104557,92,1.05,89.333996
441,BIO,ETSY,-3.936727,0.008821,0.358575,0.000460,0.438500,42.909460,64,0.80,87.215198
1045,ENPH,JNJ,-4.055910,0.005967,0.186662,0.000147,0.405688,39.557506,83,0.90,85.492083


In [10]:
sorted_trading_pairs = sorted_trading_pairs.iloc[:1500]
for i in range(len(sorted_trading_pairs)):
    trading_pair = sorted_trading_pairs.iloc[i]
    leg1 = trading_pair.loc["leg1"]
    leg2 = trading_pair.loc["leg2"]
    spread = utils.compute_spread(raw_data, leg1, leg2)
    standardized_spread = utils.zscore_normalization(spread)
    
    best_entry_threshold = trading_pair.loc["best_threshold"]
    trading_agent = PairsTradingAgent(raw_data.loc[:, leg1], raw_data.loc[:, leg2], spread)
    best_portfolio = trading_agent.trading_simulation(entry=best_entry_threshold, exit=0.5)
    
    volatility, sharpe_ratio = utils.compute_sharpe_ratio(best_portfolio, risk_free_rate=0.03)
    maximal_drawdown = utils.compute_maximal_drawdown(best_portfolio)
    if sharpe_ratio > 1.0: 
        print(leg1, leg2)
        print(f"return: {trading_pair.loc['best_return']}, sharpe ratio: {sharpe_ratio}")


AMD FAST
return: 28.82726016649396, sharpe ratio: 1.028829967686686
CDNS REGN
return: 24.137456970194695, sharpe ratio: 1.068617377327813
AMD NOW
return: 20.67571737306464, sharpe ratio: 1.0739388480578282
TECH TGT
return: 20.1060978476594, sharpe ratio: 1.254788439959022
REGN SNPS
return: 19.350555872904444, sharpe ratio: 1.0604773248523394
COR SNPS
return: 16.89122146938523, sharpe ratio: 1.0286550368460838
COR PWR
return: 13.622235381460142, sharpe ratio: 1.0260179888321046
POOL TER
return: 12.995159996570187, sharpe ratio: 1.0421993420431908
AVY MCHP
return: 12.711833596049985, sharpe ratio: 1.0677032923883598
AMD MAS
return: 11.944327175486515, sharpe ratio: 1.034609090094317
HD NDSN
return: 10.449885728437302, sharpe ratio: 1.1885580892369618
AMGN CPRT
return: 9.915183595220302, sharpe ratio: 1.0254679799365332
ADI AME
return: 9.148722728480063, sharpe ratio: 1.079624335301948
COST CTAS
return: 8.423585890077717, sharpe ratio: 1.153031406449536
AMD MSFT
return: 8.146535965352363,